In [44]:
# Import statements
import pandas as pd
import requests
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
from nba_api.stats.static import teams

In [45]:
# Define the list of 30 NBA teams in a list
nba_teams = teams.get_teams()
all_teams  = []
for team in nba_teams:
    all_teams.append(team['full_name'])

In [46]:
all_teams

['Atlanta Hawks',
 'Boston Celtics',
 'Cleveland Cavaliers',
 'New Orleans Pelicans',
 'Chicago Bulls',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Golden State Warriors',
 'Houston Rockets',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'Brooklyn Nets',
 'New York Knicks',
 'Orlando Magic',
 'Indiana Pacers',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Oklahoma City Thunder',
 'Toronto Raptors',
 'Utah Jazz',
 'Memphis Grizzlies',
 'Washington Wizards',
 'Detroit Pistons',
 'Charlotte Hornets']

In [47]:
# Extract year from the Date column of injuries
injuries = pd.read_csv('injuries.csv')
injuries = injuries.drop(['Unnamed: 0', 'Acquired'], axis=1).dropna()
injuries['Date'] = pd.to_datetime(injuries['Date'], errors='coerce')
injuries['Year'] = injuries['Date'].dt.year

# Filter by the short timeframe years that we need from 2018 to 2023
short_injuries = injuries[(injuries['Year'] >= 2018) & (injuries['Year'] <= 2023)]

short_injuries

,Date,Team,Relinquished,Notes,Year
26718,2018-01-01,Lakers,Thomas Bryant,placed on IL,2018
26720,2018-01-01,Raptors,Bruno Caboclo,placed on IL,2018
26721,2018-01-01,Raptors,Lorenzo Brown,placed on IL,2018
26723,2018-01-02,Cavaliers,Ante Zizic,placed on IL,2018
26727,2018-01-02,Kings,Frank Mason,placed on IL with bruised right heel,2018
...,...,...,...,...,...
37624,2023-04-09,Grizzlies,Ja Morant,placed on IL with right hand injury,2023
37625,2023-04-09,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow,2023
37630,2023-04-09,Kings,Davion Mitchell,placed on IL with sore left knee,2023
37631,2023-04-09,Knicks,Trevor Keels,placed on IL with illness,2023


In [64]:
# Pain Metrics

# Feature 1 - Injury Type / Severity
# Keyword mapping through feature engineering
def map_severity_score(note):
    note = str(note).lower()
    if 'surgery' in note or 'ruptured' in note:
        return 5
    elif 'torn' in note or 'acl' in note or 'achilles' in note:
        return 4
    elif 'fracture' in note or 'broken' in note:
        return 3
    elif 'strain' in note or 'sprain' in note:
        return 2
    elif 'sore' in note or 'tightness' in note:
        return 1
    else:
        return 1

short_injuries['Severity'] = short_injuries['Notes'].apply(map_severity_score)

# Feature 2 - Position / Star Impact
# Count how many times each player appears in the 'Relinquished' column
player_freq = short_injuries['Relinquished'].value_counts()

# If player appears more than usual, assume star player because of the notion that they play frequently on their respective team
def estimate_impact(player):
    count = player_freq.get(player, 1)
    if count >= 15:
        return 3  
    elif count >= 5:
        return 2  
    else:
        return 1  

short_injuries['PlayerImpact'] = short_injuries['Relinquished'].apply(estimate_impact)
short_injuries


/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_35200/280422089.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_injuries['Severity'] = short_injuries['Notes'].apply(map_severity_score)
/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_35200/280422089.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_injuries['PlayerImpact'] = short_injuries['Relinquished'].apply(estimate_impact)


,Date,Team,Relinquished,Notes,Year,Severity,PlayerImpact
26718,2018-01-01,Lakers,Thomas Bryant,placed on IL,2018,1,2
26720,2018-01-01,Raptors,Bruno Caboclo,placed on IL,2018,1,1
26721,2018-01-01,Raptors,Lorenzo Brown,placed on IL,2018,1,2
26723,2018-01-02,Cavaliers,Ante Zizic,placed on IL,2018,1,2
26727,2018-01-02,Kings,Frank Mason,placed on IL with bruised right heel,2018,1,2
...,...,...,...,...,...,...,...
37624,2023-04-09,Grizzlies,Ja Morant,placed on IL with right hand injury,2023,1,3
37625,2023-04-09,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow,2023,1,3
37630,2023-04-09,Kings,Davion Mitchell,placed on IL with sore left knee,2023,1,1
37631,2023-04-09,Knicks,Trevor Keels,placed on IL with illness,2023,1,1
